# Testing YOLOv8
the test would be YOLOv8 performing detection of an image, and then on a live webcam
Note: the detection shown would only be "person" detection, which is box.cls[0]

In [1]:
from ultralytics import YOLO
import cv2
import time
import numpy as np
import os
from insightface import app
from scipy.spatial.distance import cosine

In [2]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.10.0 =====================================
  Version control:               4.10.0

  Platform:
    Timestamp:                   2024-06-17T17:58:36Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (8 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (36 files):           + SSSE3 SSE4_

In [3]:
model = YOLO('yolov8n.pt')
img = 'kantor(10.jpeg'
results = model(img)

results


image 1/1 d:\Kuliah\7th sem\yolo_test\kantor(10.jpeg: 480x640 15 persons, 1 backpack, 1 handbag, 3 bottles, 2 cups, 1 bowl, 11 chairs, 10 laptops, 1 book, 118.0ms
Speed: 4.2ms preprocess, 118.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [4]:
# show the entire frame

image = cv2.imread(img)

for box in results[0].boxes:
    cls = int(box.cls[0])
    if cls != 0:
        continue

    x1, y1, x2, y2 = map(int, box.xyxy[0])
    conf = box.conf[0]
    label = f"{results[0].names[cls]} {conf:.2f}"

    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Box in green
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imshow("YOLO Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# show how would the cropped faces look

image = cv2.imread(img)

for i, box in enumerate(results[0].boxes):
    cls = int(box.cls[0])  # Class index
    if cls != 0:
        continue

    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped_object = image[y1:y2, x1:x2]
    
    # Save or display the cropped object
    cv2.imshow(f"Cropped Person {i}", cropped_object)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [17]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

prev_time = 0
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    current_time = time.time()

    results = model(frame)  # YOLO inference on the current frame

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        cls = int(box.cls[0])
        label = f"{results[0].names[cls]}"

        if cls != 0:
            continue

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Box in green
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    fps_text = f"FPS: {fps:.2f}"
    cv2.putText(frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    
    cv2.imshow("YOLOv8 Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 2 persons, 74.0ms
Speed: 3.0ms preprocess, 74.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 57.0ms
Speed: 1.0ms preprocess, 57.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 54.0ms
Speed: 1.0ms preprocess, 54.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 55.0ms
Speed: 1.0ms preprocess, 55.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 56.0ms
Speed: 1.0ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 56.0ms
Speed: 1.0ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 56.0ms
Speed: 2.0ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 53.0ms
Speed: 1.0ms preprocess, 53.0ms inference, 0.0ms postprocess per image at shape (

# Implementing InsightFace
let's see how slow this thing is 💀

In [16]:
# Load the InsightFace model
face_recognition = app.FaceAnalysis()
face_recognition.prepare(ctx_id=0, det_size=(640, 640))

# Load the face database
database_path = 'Database'  # Replace with the path to your database
face_embeddings = {}
for person_name in os.listdir(database_path):
    person_folder = os.path.join(database_path, person_name)
    if os.path.isdir(person_folder):
        face_embeddings[person_name] = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                faces = face_recognition.get(img)
                for face in faces:
                    # Store the normalized embedding of the first face detected
                    face_embeddings[person_name].append(face.normed_embedding)


c:\Users\Deciano\.virtualenvs\yolo_test-9WuQH8tP\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112,

In [3]:
model = YOLO('yolov8n.pt')

In [14]:
cap = cv2.VideoCapture(0)

In [19]:
cap = cv2.VideoCapture("vid_kantor.mp4")

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

prev_time = 0
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    current_time = time.time()

    results = model(frame)  # YOLO inference on the current frame

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        cls = int(box.cls[0])
        label = f"{results[0].names[cls]}"

        if cls != 0:
            continue

        cur_person = frame[y1:y2, x1:x2]
        faces = face_recognition.get(cur_person)

        for face in faces:
            bbox = face.bbox.astype(int)
            identity = face.normed_embedding

            recognized_person = None
            min_distance = float('inf')
            for person_name, embeddings in face_embeddings.items():
                for embedding in embeddings:
                    distance = cosine(identity, embedding)
                    if distance < min_distance:
                        min_distance = distance
                        recognized_person = person_name

            if recognized_person is not None and min_distance < 0.8:  # Adjust threshold as necessary
                face_label = recognized_person
            else:
                face_label = "Unknown"
            cv2.putText(frame, face_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Box in green
        # cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    fps_text = f"FPS: {fps:.2f}"
    cv2.putText(frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    
    cv2.imshow("YOLOv8 Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 15 persons, 1 handbag, 2 cups, 6 chairs, 4 laptops, 1 cell phone, 119.0ms
Speed: 5.0ms preprocess, 119.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


c:\Users\Deciano\.virtualenvs\yolo_test-9WuQH8tP\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 384x640 11 persons, 1 handbag, 2 cups, 6 chairs, 4 laptops, 1 cell phone, 63.0ms
Speed: 2.0ms preprocess, 63.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 handbag, 1 cup, 6 chairs, 4 laptops, 1 cell phone, 66.3ms
Speed: 2.0ms preprocess, 66.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 handbag, 2 cups, 8 chairs, 4 laptops, 1 cell phone, 1 book, 59.0ms
Speed: 2.0ms preprocess, 59.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 handbag, 1 cup, 7 chairs, 5 laptops, 1 cell phone, 1 book, 48.0ms
Speed: 1.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 1 handbag, 1 cup, 7 chairs, 5 laptops, 1 cell phone, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 handbag, 1 cup, 9 chairs, 5 laptops, 1 cell phone, 